## Домашнее задание к лекции «Поиск выбросов и генерация новых признаков»

Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет www.kaggle.com...iques/data (train.csv)

Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей

Построить случайный лес, вывести важность признаков

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии

Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)


In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, roc_curve, roc_auc_score

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
#Находим категориальные признаки
cat_feat = list(data.dtypes[data.dtypes == object].index)

#закодируем пропущенные значения строкой, факт пропущенного значения тоже может нести в себе информацию
data[cat_feat] = data[cat_feat].fillna('nan')

In [5]:
X = data.drop(['SalePrice'], axis = 1)
y = data['SalePrice']

In [6]:
# переводим категориальные переменные в one-hot encoding

X1 = X
X1 = pd.get_dummies(X1, columns=cat_feat)

In [7]:
# Заполняем числовые пропуски средними значениями для колонок

X1['LotFrontage'] = X1['LotFrontage'].fillna(X1['LotFrontage'].mean())
X1['MasVnrArea'] = X1['MasVnrArea'].fillna(X1['MasVnrArea'].mean())
X1['GarageYrBlt'] = X1['GarageYrBlt'].fillna(X1['GarageYrBlt'].mean())

In [15]:
# Стандартизируем данные 
scaler = StandardScaler()
scaler.fit(X1)
scaled_values = scaler.transform(X1)
X2 = pd.DataFrame(data=scaled_values, columns=X1.columns)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state = 31)

In [17]:
# Строим случайный лес

reg_rf = RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)
reg_rf.fit(X_train, y_train)   

RandomForestRegressor(max_depth=5, max_features=0.5, min_samples_leaf=20,
                      n_estimators=10, n_jobs=-1)

In [18]:
# оцениваем точность модели случайного леса с помощью кросс-валидации

print(cross_val_score(reg_rf, X_train, y_train, cv=10))
cross_val_score(reg_rf, X_train, y_train, cv=10).mean()

[0.86311855 0.8814226  0.80871851 0.67757649 0.8065535  0.78114449
 0.77699746 0.73858162 0.8734116  0.76658317]


0.7865878393088523

In [19]:
# Выводим значимость фичей

a = pd.DataFrame(data=np.append([X1.columns],[reg_rf.feature_importances_], axis =0).T, columns=["Feature", "Importance"])
a.sort_values(by=['Importance'], ascending=False).head(15)

,Feature,Importance
4,OverallQual,0.467908
26,GarageCars,0.163025
16,GrLivArea,0.11578
185,BsmtQual_Ex,0.050172
12,TotalBsmtSF,0.03368
27,GarageArea,0.02398
19,FullBath,0.020954
173,ExterQual_TA,0.017222
233,KitchenQual_Ex,0.015683
13,1stFlrSF,0.013727


In [20]:
# Обучаем стекинг на трех моделях

estimators = [
    ('lr', LinearRegression()),
    ('svr', svm.SVR()),
    ('dt', DecisionTreeRegressor())
]

In [21]:
reg1 = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,random_state=42)
)

In [22]:
reg1.fit(X_train, y_train)

StackingRegressor(estimators=[('lr', LinearRegression()), ('svr', SVR()),
                              ('dt', DecisionTreeRegressor())],
                  final_estimator=RandomForestRegressor(n_estimators=10,
                                                        random_state=42))

In [23]:
# Оцениваем точность стэкинг модели

print(cross_val_score(reg1, X_train, y_train, cv=10))
cross_val_score(reg1, X_train, y_train, cv=10).mean()

[0.90120698 0.81735588 0.82078211 0.6516429  0.89638351 0.85905443
 0.8401724  0.83852122 0.82480485 0.88583204]


0.8336772545851321

In [57]:
# Сравнимваем на холд-ауте. SME стекинга модели меньше, чем у случайного леса

y_pred_rf = reg_rf.predict(X_test)
y_pred_stack = reg1.predict(X_test)

print('MSE for single random forest:', mean_squared_error(y_test, y_pred_rf))
print('Stacking mse', mean_squared_error(y_test, y_pred_stack))

MSE for single random forest: 1338652346.9189978
Stacking mse 1194403687.6219406


-144248659.29705715